In [ ]:
using JuMP
using Gurobi, Ipopt
using PyPlot
using LinearAlgebra
using CSV
using DataFrames
using NonlinearSolve
using DataStructures

In [ ]:
# read data
l1 = DataFrame(CSV.File("data\\top_2.csv"))
lm = DataFrame(CSV.File("data\\middle_2.csv"))
l3  = DataFrame(CSV.File("data\\bottom_2.csv"))

In [ ]:
N = 50
lx = l1[!,1][1:N]
ly = l1[!,2][1:N]

rx = l3[!,1][1:N]
ry = l3[!,2][1:N]

mx = lm[!,1][1:N]
my = lm[!,2][1:N]

cx1 = (lx + mx)/2
cy1 = (ly + my)/2

cx2 =  (rx + mx)/2
cy2 =  (ry + my)/2

ox = cx2[20]
oy = cy2[20]

plot(lx,ly)
plot(rx,ry)
plot(mx,my)
plot(cx1,cy1)
plot(cx2, cy2)
scatter(ox,oy)

In [ ]:
cx = cx2[1:1]
append!(cx, cx1[2:end])
cy = cy2[1:1]
append!(cy, cy1[2:end])
plot(lx,ly)
plot(rx,ry)
plot(mx,my)
scatter(cx,cy)

cx = cx1
cy = cy1

In [ ]:
# function E_cur(i)
#     i = trunc(Int, i)
#     if i == 1
#         v1 = X[i,:]
#         v2 = X[i+1,:] - X[i,:]
#         norm_v1 = v1'*v1
#         norm_v2 = v2'*v2
#         dTheta = acos( dot(v1,v2)/ (norm_v1*norm_v2))
#         deltaX = X[i,:]'*X[i,:]
#         return dTheta/deltaX
#     elseif i == N
#         v1 = X[i,:] - X[i-1,:]
#         v2 = -X[i,:]
#         norm_v1 = v1'*v1
#         norm_v2 = v2'*v2
#         v1v2 = dot(v1,v2)
#         #dTheta = acos( dot(v1,v2)/ (norm_v1*norm_v2))
#         @NLexpression(m, dTheta,  acos(v1v2/ (norm_v1*norm_v2)))
#         dx_norm =  (X[i,:] - X[i-1,:])'*(X[i,:] - X[i-1,:]) 
#         @NLexpression(m, deltaX, dx_norm)
#         return dTheta/deltaX
#     else
#         v1 = X[i,:] - X[i-1,:]
#         v2 = X[i+1,:] - X[i,:]
#         norm_v1 = v1'*v1
#         norm_v2 = v2'*v2
        
#         dTheta = acos( dot(v1,v2)/ (norm_v1*norm_v2))
#         deltaX = (X[i,:] - X[i-1,:])'*((X[i,:] - X[i-1,:]))
#         return dTheta/deltaX       
#     end
# end

In [ ]:
function E_cur_i(xi,yi,xprev,yprev,xnext,ynext)
    E_cur = 0.0
    v1x = xi - xprev
    v1y = yi - yprev
    
    v2x = xnext - xi
    v2y = ynext - yi

    v1v2 = v1x*v2x + v1y*v2y
    v1norm = sqrt(v1x^2 + v1y^2)
    v2norm = sqrt(v2x^2 + v2y^2)
    
    if v1norm > 0 && v2norm > 0
        frac = v1v2/(v1norm*v2norm)
        deltaTheta = 0
        if frac < 1
            deltaTheta = acos(frac)
        end
        deltaX = v1norm*v2norm
        E_cur = deltaTheta/deltaX
    end
    return E_cur
end

In [ ]:
function E_dev_i(x,y,cx,cy)
    dx = (x - cx)^2
    dy = (y - cy)^2
    #println(dx + dy)
    return dx + dy
end

In [ ]:
function E_obs_i(xi,yi,ox,oy)
    thresh = 0.5
    dxo = (xi - ox)^2
    dyo = (yi - oy)^2
    dobsnorm = sqrt(dxo + dyo)
    
    E_obs = 0.0
    if dobsnorm < thresh
        if dobsnorm > 0
            E_obs = ( (thresh/dobsnorm)  - 1)^2
        else
            E_obs = 250000
        end
    end
    
    return E_obs
end

In [ ]:
f(x) = (0.5/x - 1)^2
x = range(0.001,0.5,1000)
y = f.(x)
plot(x,y)

In [ ]:
function E_i(xi,yi,xprev,yprev,xnext,ynext,cx,cy,ox,oy)
    
#     # curvature term
#     v1x = xi - xprev
#     v1y = yi - yprev
    
#     v2x = xnext - xi
#     v2y = ynext - yi

#     v1v2 = v1x*v2x + v1y*v2y
#     v1norm = sqrt(v1x^2 + v1y^2)
#     v2norm = sqrt(v2x^2 + v2y^2)
    
#     deltaTheta = acos(v1v2/(v1norm*v2norm))
#     deltaX = v1norm
    
#     E_cur = deltaTheta/deltaX
    
    # Deviation term
    dx = (xi - cx)^2
    dy = (yi - cy)^2
    
    E_dev = (dx + dy)
    
#     # Obstacle term
#     thresh = 0.5
#     dxo = (xi - ox)^2
#     dyo = (yi - oy)^2
#     dobsnorm = sqrt(dxo + dyo)
    
#     E_obs = 0
#     if dobsnorm < thresh
#         E_obs = ((1/dobsnorm) - 1)^2
#     end
    #println("Hello", E_dev)
    return E_dev
end

In [ ]:
function ∇E_i()

In [ ]:
m = Model(Ipopt.Optimizer)
register(m, :E_cur_i, 6, E_cur_i, autodiff=true)
register(m, :E_dev_i, 4, E_dev_i, autodiff=true)
register(m, :E_obs_i, 4, E_obs_i, autodiff=true)
#register(m, :E_i, 10, E_i, autodiff=true)
w_cur = 1
w_dev = 1
w_obs = 1
@variable(m,x[1:N])
@variable(m, y[1:N])
set_start_value.(x, cx)
set_start_value.(y, cy)


#delta_theta = [ (x[i] - x[i+1]) for i = 1:99]
@NLexpression(m, E_cur, sum(E_cur_i(x[i],y[i], x[i-1],y[i-1], x[i+1],y[i+1]) for i in 2:N-1))
@NLexpression(m, E_dev, sum(E_dev_i(x[i],y[i], cx[i], cy[i]) for  i in 2:N-1))
#@NLexpression(m, E_obs, sum(E_obs_i(x[i],y[i],ox,oy)  for i in 2:N-1))
#@NLexpression(m, E_cur, sum(  acos( dot((X[i,:] - X[i-1,:]), (X[i+1,:] - X[i,:]))/(  ((X[i,:] - X[i-1,:])'*(X[i,:] - X[i-1,:]))*((X[i+1,:] - X[i,:])'*(X[i+1,:] - X[i,:])) )) /  ((X[i,:] - X[i-1,:])'*((X[i,:] - X[i-1,:])))    for i in 2:99) )
@NLobjective(m, Min, E_cur + E_dev)

#@NLobjective(m, Min, sum(E_i(x[i],y[i], x[i-1],y[i-1], x[i+1],y[i+1],cx[i],cy[i], ox,oy) for i in 2:99 ))
#print(m)

optimize!(m)

In [ ]:
xopt = value.(x)
yopt = value.(y)

In [ ]:
#plot(lx,ly)
#plot(rx,ry)
#plot(cx,cy)
plot(xopt,yopt)

In [ ]:
mutable struct PIDController
    Kp::Float64
    Ki::Float64
    Kd::Float64
    setpoint::Float64
    last_error::Float64
    integral::Float64
end

function PIDController(Kp::Float64, Ki::Float64, Kd::Float64, setpoint::Float64)
    return PIDController(Kp, Ki, Kd, setpoint, 0.0, 0.0)
end

function update!(pid::PIDController, measured_value::Float64, dt::Float64)
    error = pid.setpoint - measured_value
    pid.integral += error * dt
    derivative = (error - pid.last_error) / dt
    output = pid.Kp * error + pid.Ki * pid.integral + pid.Kd * derivative
    pid.last_error = error

    return output
end


In [ ]:
function simulate_system(pid_controller::PIDController, plant, timesteps, dt)
    system_state = 0.0
    control_input_history = []
    system_state_history = []

    for t in 1:timesteps
        control_input = update!(pid_controller, system_state, dt)
        push!(control_input_history, control_input)

        system_state = plant(system_state, control_input, dt)
        push!(system_state_history, system_state)
    end

    return system_state_history, control_input_history
end

# Define a simple plant model
function plant(state, control_input, dt)
    return state + control_input * dt
end

# Set up PID controller
Kp = 1.0
Ki = 0.5
Kd = 0.1
setpoint = 5.0
pid_controller = PIDController(Kp, Ki, Kd, setpoint)

# Simulate the system
timesteps = 100
dt = 0.1
system_state_history, control_input_history = simulate_system(pid_controller, plant, timesteps, dt)

# Print the results
println("System state history: ", system_state_history)
println("Control input history: ", control_input_history)


In [ ]:
ref_pts  = DataFrame(CSV.File("data\\input_trajectory.csv"))
cx = ref_pts[!,1]
cy = ref_pts[!,2]

In [ ]:


mutable struct BicycleModel
    x::Float64
    y::Float64
    yaw::Float64
    v::Float64
    L::Float64
end

function update!(bicycle::BicycleModel, delta::Float64, a::Float64, dt::Float64)
    bicycle.x += bicycle.v * cos(bicycle.yaw) * dt
    bicycle.y += bicycle.v * sin(bicycle.yaw) * dt
    bicycle.yaw += bicycle.v / bicycle.L * tan(delta) * dt
    bicycle.v += a * dt
end

function read_reference_trajectory(file_path::String)
    trajectory_data = CSV.read(file_path, DataFrame)
    return trajectory_data
end

x_pts  = []
y_pts = []
function main()
    # Read reference trajectory from a CSV file
    #file_path = "reference_trajectory.csv"
    #reference_trajectory = read_reference_trajectory(file_path)

    # Initialize the bicycle model
    bicycle = BicycleModel(0.0, 0.0, 0.0, 0.0, 2.0)

    # Set up PID controllers for x and y positions
    Kp_x = 1.0
    Ki_x = 0.5
    Kd_x = 0.1
    Kp_y = 1.0
    Ki_y = 0.5
    Kd_y = 0.1
    pid_x = PIDController(Kp_x, Ki_x, Kd_x, 0.0)
    pid_y = PIDController(Kp_y, Ki_y, Kd_y, 0.0)

    # Simulation parameters
    timesteps = size(cx, 1)
    dt = 0.1
    
    for t in 1:timesteps
        # Update setpoints for PID controllers
        pid_x.setpoint = cx[t]#reference_trajectory[t, "x"]
        pid_y.setpoint = cy[t]#reference_trajectory[t, "y"]

        # Calculate control inputs using PID controllers
        a_x = update!(pid_x, bicycle.x, dt)
        a_y = update!(pid_y, bicycle.y, dt)

        # Calculate steering angle and acceleration inputs for the bicycle model
        delta = atan(a_y, a_x)
        a = sqrt(a_x^2 + a_y^2)

        # Update the bicycle model
        update!(bicycle, delta, a, dt)
        append!(x_pts, bicycle.x)
        append!(y_pts, bicycle.y)
        # Print the bicycle state at each time step
        println("Step: $t, x: $(bicycle.x), y: $(bicycle.y), yaw: $(bicycle.yaw), v: $(bicycle.v)")
    end
end

main()


In [ ]:
#plot(cx,cy)
scatter(x_pts,y_pts)

In [ ]:
y_pts